In [1]:
import pandas as pd

path = 'OneDrive_1_14/S_Data/'
endpoint = '.csv'

csvs = ['customers', 
        'geolocation', 
        'order_items', 
        'order_payments', 
        'order_reviews', 
        'orders', 
        'product_category_name_translation',
        'products',
        'sellers']
dfs = {}

for name in csvs:
    print(name.upper(),'\n\n')
    dfs[name] = pd.read_csv(path+name+endpoint)
    

CUSTOMERS 


GEOLOCATION 


ORDER_ITEMS 


ORDER_PAYMENTS 


ORDER_REVIEWS 


ORDERS 


PRODUCT_CATEGORY_NAME_TRANSLATION 


PRODUCTS 


SELLERS 




# Task 1

### To do

#### get reviews in function same as other values in other df then merge dfs

##### (done) english name, 
##### (done) average delivery time, 
##### (done) average delivery time from last 14 days,
##### (done) average product price,
##### (done) actual product price,
##### (done) sales from last 14 days, 
##### (done) total sale per seller from 14 days, 
##### reviews since products sale started, 
##### number of reviews added in 14 days, 
##### average fracht_val in past 14 days

### Code

#### 1. English name

In [2]:
dfs['products'].head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [3]:
dfs['product_category_name_translation'].head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [4]:
## get english category name

dfs['products'] = dfs['products'].merge(dfs['product_category_name_translation'],
                                        how = 'inner',
                                        on = 'product_category_name')

#### 2. Add missing delivery dates, get necessary date for subset

In [5]:
dfs['orders'].head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [6]:
dfs['orders']['order_status'].unique()

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

In [7]:
## get only valid orders
dfs['orders'] = dfs['orders'].loc[(dfs['orders']['order_status'] != 'canceled') & 
                                  (dfs['orders']['order_status'] != 'unavailable')]

In [8]:
print(dfs['orders'].isna().sum())

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                  19
order_delivered_carrier_date      624
order_delivered_customer_date    1737
order_estimated_delivery_date       0
dtype: int64


In [9]:
print(dfs['orders'].dtypes,
      '\n', 
      dfs['order_reviews'].dtypes)

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object 
 review_id                  object
order_id                   object
review_score                int64
review_comment_title       object
review_comment_message     object
review_creation_date       object
review_answer_timestamp    object
dtype: object


In [10]:
## merge dataframes for product id, price, number of ordered items, reviews
dfs['orders'] = dfs['order_items'].merge(dfs['orders'], on = 'order_id', how = 'right')
dfs['orders'] = dfs['orders'].merge(dfs['order_reviews'], on='order_id', how ='left')
dfs['orders'] = dfs['orders'].merge(dfs['products'], on = 'product_id', how = 'left')

In [11]:
dfs['orders']['order_purchase_timestamp'] = pd.to_datetime(dfs['orders']['order_purchase_timestamp'])
dfs['orders']['order_delivered_customer_date'] = pd.to_datetime(dfs['orders']['order_delivered_customer_date'])
dfs['orders']['review_creation_date'] = pd.to_datetime(dfs['orders']['review_creation_date'])

## fill nan values after grouping by product id so values are more reliable
dfs['orders'].sort_values(by='product_id', inplace=True)
dfs['orders']['order_delivered_customer_date'].ffill(inplace=True)

/var/folders/bz/m_9l75x14kz_37vfj8r34b2r0000gn/T/ipykernel_17079/1906626660.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfs['orders']['order_delivered_customer_date'].ffill(inplace=True)


In [12]:
## get order date, review date, delivery date and delivery time

dfs['orders']['purchase_year'] = dfs['orders']['order_purchase_timestamp'].dt.year
dfs['orders']['purchase_day'] = dfs['orders']['order_purchase_timestamp'].dt.dayofyear

dfs['orders']['review_year'] = dfs['orders']['review_creation_date'].dt.year
dfs['orders']['review_day'] = dfs['orders']['review_creation_date'].dt.dayofyear


dfs['orders']['delivery_year'] = dfs['orders']['order_delivered_customer_date'].dt.year
dfs['orders']['delivery_day'] = dfs['orders']['order_delivered_customer_date'].dt.dayofyear

## calculate delivery time (check if delivery and purchase were made in same year)
dfs['orders']['delivery_time'] = ((dfs['orders']['delivery_year'] - dfs['orders']['purchase_year'])*365 + 
                                 dfs['orders']['delivery_day'] - dfs['orders']['delivery_day'])

## if delivery and purchase years were different check if purchase year was leap
for index, row in dfs['orders'].iterrows():
    if row['purchase_year'] != row['delivery_year']:
        if row['purchase_year']%4 ==0 and row['purchase_year']%100 == 0 and row['purchase_year']%400 == 0:
            dfs['orders'].at[index, 'delivery_time'] += 1

#### 3.

In [13]:
## get data from 14 days
def is_leap(year):
    if year%4 == 0 and year%100 == 0 and year%400 == 0:
        return True
    else:
        return False

In [14]:
## get end of subset (check if it ends in following year)
def get_end(year, day_s, sub_size):
    
    if is_leap(year):
        if day_s > 366 - sub_size:
            day_e = 366 - day_s
            year_e = year + 1
        else:
            day_e = day_s + sub_size
            year_e = year
    else:
        if day_s > 365 - sub_size:
            day_e = 365 - day_s
            year_e = year + 1
        else:
            day_e = day_s + sub_size
            year_e = year
    return day_s, year

In [ ]:
def get_historical_data(dfs, day_end, year_end, column_gb):
    
    ## income from product per order
    dfs['orders']['incomes'] = dfs['orders']['order_item_id']*dfs['orders']['price']

    ## orders till end date
    orders_bef = dfs['orders'].loc[(dfs['orders']['purchase_year'] < year_end) |
                                   ((dfs['orders']['purchase_year'] == year_end) &
                                    (dfs['orders']['purchase_day'] < day_end))]
    ## df with orders before end day
    df_bef = orders_bef.groupby(column_gb).agg({'delivery_time': 'mean', 
                                                                      'price': 'mean', 
                                                                      'order_item_id': 'sum', 
                                                                      'incomes': 'sum',
                                                                      'freight_value': 'mean'}).reset_index()    
    ## df with orders per seller before end day 
    df_bef_per_seller = orders_bef.groupby([column_gb, 'seller_id']).agg({'delivery_time': 'mean', 
                                                                                                'price': 'mean', 
                                                                                                'order_item_id': 'sum',
                                                                                                'incomes': 'sum',
                                                                                                'freight_value': 'mean'}).reset_index()
    ## reviews since sale began
    review_bef = dfs['orders'].loc[(dfs['orders']['review_year'] < year_end) |
                                   ((dfs['orders']['review_year'] == year_end) &
                                    (dfs['orders']['review_day'] < day_end))]
    
    ## df with mean reviews score for each product category since it sales began
    df_bef_reviews = review_bef.groupby(column_gb)['review_score'].mean().reset_index()

    ## merging dataframes to get proper information about product
    df_before_start_day = df_bef.merge(df_bef_reviews, on = column_gb, how = 'left')
    df_before_start_day.rename(columns={'delivery_time': 'avg_delivery_time', 
                                        'price': 'avg_price',
                                        'order_item_id' : 'number_of_sold_items',
                                        'freight_value': 'avg_freight_val',
                                        'review_score': 'avg_review_score'},
                               inplace = True)
    
    ## merging dataframes with collected informations for each seller
    df_before_start_day_per_seller = df_bef_per_seller.merge(df_bef_reviews, on = column_gb, how = 'left')
    df_before_start_day_per_seller.rename(columns={'delivery_time': 'avg_delivery_time', 
                                        'price': 'avg_price',
                                        'order_item_id' : 'number_of_sold_items',
                                        'freight_value': 'avg_freight_val',
                                        'review_score': 'avg_review_score'},
                               inplace = True)

## at the end we have 2 dataframes: 
## 1st with product information since it sales began to start day
## 2nd with product information same as in 1st one but for each seller
    return(df_before_start_day, df_before_start_day_per_seller)

In [21]:
def get_subset_data(dfs, year, day_s, sub_size, column_gb):
    
    ##calculate end of subset
    day_e, year_e = get_end(year, day_s, sub_size)

    ## income from product per order
    dfs['orders']['incomes'] = dfs['orders']['order_item_id']*dfs['orders']['price']
    
    ## orders from start date to end date
    if year == year_e:
        orders_act = dfs['orders'].loc[(dfs['orders']['purchase_year'] == year) & 
                                       (dfs['orders']['purchase_day'] >= day_s) & 
                                       (dfs['orders']['purchase_day'] <= day_e)]
    else:
        orders_act = dfs['orders'].loc[((dfs['orders']['purchase_year'] == year) & 
                                        (dfs['orders']['purchase_day'] >= day_s)) |
                                       ((dfs['orders']['purchase_year'] == year_e) & 
                                        (dfs['orders']['purchase_day'] <= day_e))]
                                        
## each dataframe consists 
##    mean values: delivery time, price and freight value
##    sum values: incomes and number of ordered items

    ## df with orders from start date to end date
    df_act = orders_act.groupby(column_gb).agg({'delivery_time': 'mean', 
                                                   'price': 'mean', 
                                                   'order_item_id': 'sum', 
                                                   'incomes': 'sum',
                                                   'freight_value': 'mean'}).reset_index()
                                
## same information collected for each seller

    ## df with orders from start date to end date
    df_act_per_seller = orders_act.groupby([column_gb, 'seller_id']).agg({'delivery_time': 'mean', 
                                                                             'price': 'mean', 
                                                                             'order_item_id': 'sum',
                                                                             'incomes': 'sum',
                                                                             'freight_value': 'mean'}).reset_index()

    ## reviews added between start and end date 
    if year == year_e:
        review_act = dfs['orders'].loc[(dfs['orders']['review_year'] == year) & 
                                       (dfs['orders']['review_day'] >= day_s) & 
                                       (dfs['orders']['review_day'] <= day_e)]
    else:
        review_act = dfs['orders'].loc[((dfs['orders']['review_year'] == year) & 
                                        (dfs['orders']['review_day'] >= day_s)) |
                                       ((dfs['orders']['review_year'] == year_e) & 
                                        (dfs['orders']['review_day'] <= day_e))]

    ## df with mean reviews score value in time period from start to end date for each product 
    df_act_reviews = review_act.groupby(column_gb)['review_score'].mean().reset_index()

## merging dataframes to get proper information about product
    df_actual = df_act.merge(df_bef_reviews, on = column_gb, how = 'left')
    df_actual.rename(columns={'delivery_time': 'avg_delivery_time', 
                              'price': 'avg_price',
                              'order_item_id' : 'number_of_sold_items',
                              'freight_value': 'avg_freight_val',
                              'review_score': 'avg_review_score'},
                     inplace = True)                     

## merging dataframes with collected informations for each seller
    df_actual_per_seller = df_act_per_seller.merge(df_bef_reviews, on = column_gb, how = 'left')
    df_actual_per_seller.rename(columns={'delivery_time': 'avg_delivery_time', 
                                        'price': 'avg_price',
                                        'order_item_id' : 'number_of_sold_items',
                                        'freight_value': 'avg_freight_val',
                                        'review_score': 'avg_review_score'},
                               inplace = True)

## at the end we have 2 dataframes: 
## 1st with product information from start date to end date
## 2nd with product information same as in 1st one but for each seller
    return(df_actual, df_actual_per_seller)

In [ ]:
## calculate period of time for historical data
def period(day, year, year_start, day_start):
    if year == year_start:
        return day - day_start
    elif year - year_start == 1:
        if is_leap(year_start):
            return day + 366 - day_start
        else:
            return day + 365 - day_start
    else:
        if is_leap(year_start):
            days = 366 - day_start
        else:
            days = 365 - day_start
        for i in range(year-year_start-1):
            if is_leap(year_start+i+1):
                days+=366
            else:
                days+=365
        return days

In [ ]:
def add_dates_subset(df, day_s, year_s, sub_size):
    day_e, year_e = get_end(year, day_s, sub_size)
    df['start_day'] = day_start
    df['start_year'] = year_start
    df['time_period'] = sub_size
    df['end_day'] = day_e
    df['end_year'] = year_e
    return df

In [ ]:
def add_dates_historical(df, day_start, year_start, day_end, year_end):
    df['start_day'] = day_start
    df['start_year'] = year_start
    df['time_period'] = period(day_end, year_end)
    df['end_day'] = day_end
    df['end_year'] = year_end
    return df

In [22]:
subset_14(dfs, year_start, day_start+100, subset_size)

Empty DataFrame
Columns: [product_category, avg_delivery_time, avg_price, number_of_sold_items, incomes, avg_freight_val, avg_review_score]
Index: []


(                  product_category  avg_delivery_time   avg_price  \
 0                 air_conditioning         146.000000  170.709000   
 1                            audio           0.000000   78.495000   
 2                             auto          81.111111  156.472222   
 3                             baby           0.000000  124.651818   
 4                   bed_bath_table           0.000000   59.873750   
 5           books_general_interest         365.000000  119.500000   
 6                  books_technical         365.000000  267.000000   
 7            computers_accessories         191.190476   66.634286   
 8                   consoles_games          73.000000  388.226000   
 9                       cool_stuff           0.000000  149.442857   
 10             diapers_and_hygiene           0.000000  134.900000   
 11                     electronics           0.000000  107.990000   
 12          fashio_female_clothing           0.000000   54.900000   
 13        fashion_b

In [17]:
year_start = dfs['orders'].sort_values(by='purchase_year')['purchase_year'].values[0]
day_start =  dfs['orders'].sort_values(by=['purchase_year', 'purchase_day'])['purchase_day'].values[0]

year_end =  dfs['orders'].sort_values(by='purchase_year', ascending = False)['purchase_year'].values[0]
day_end =  dfs['orders'].sort_values(by=['purchase_year', 'purchase_day'], 
                                     ascending = [False, False])['purchase_day'].values[0]
if day_end >= 7:
    day_end -= 7
else:
    year_end -= 1
    day_end = 365 + day_end - 7
    if is_leap(year_end):
        day_end += 1
    else:
        pass

offset = 0 # after how many subset collected  you want to start collecting historical data (from start day there wont be much data)
subset_size = 14
loop = 0
start = True
is_product_new = False


if is_product_new:
    column_groupby = 'product_id'
else:
    column_groupby = 'product_category_name_english'

data = pd.DataFrame()
data_per_seller = pd.DataFrame()

## go through all years and days, start parameter let start from proper day
for year in range(year_start, year_end +1):
    for day in range(0, 365):
        loop += 1
        if start:
            day = day_start
            start = False
            pass
        else:
            if (day >= day_end)&(year>=year_end):
                break
            else:
                actual, actual_per_seller = get_subset_data(dfs, year, day, subset_size, columns_groupby)
                actual = add_dates_subset(actual, day_start, year_start, subset_size)
                actual_per_seller = add_dates_subset(before_per_seller, day_start, year_start, subset_size)
                if loop == 1:
                    data = actual
                    data_per_seller = actual_per_seller
                else:
                    data = pd.concat([data, actual], ignore_index=True)
                    data_per_seller = pd.concat([data_per_seller, actual_per_seller], ignore_index=True)
                day = day + subset_size
                
historical, historical_per_seller =  get_historical_data(dfs, day_end, year_end, columns_groupby)
historical = add_dates_historical(historical, day_start, year_start, day_end, year_end)
historical_per_seller = add_dates_historical(historical_per_seller, day_start, year_start, day_end, year_end)
data = pd.concat([data, historical], ignore_index=True)
data_per_seller = pd.concat([data_per_seller, historical_per_seller], ignore_index=True)

end


### TIME FOR TRAIN AND TEST 
### ALSO CHECK IF NEW PRODUCT IF YES FIND SIMILIAR